In [14]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

In [ ]:
def Chem_generator(smiles):
    res_list = []
    for i in tqdm(smiles):
        mol = Chem.MolFromSmiles(i)
        temp = []
        index = 0
        while(len(set(temp))!=4 and index != 100):
            index+=1
            temp.append(Chem.MolToSmiles(mol,doRandom=True))
        res_list+=list(set(temp))
    return res_list

In [15]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
import atomInSmiles
from tqdm import tqdm
import atomInSmiles
def encode(smiles_list,labels):
    mols = []
    res_labels = []
    len_list = []
    
    for smiles in smiles_list:
        mol = MolFromSmiles(smiles)
        mols.append(mol)
    res = []
    for index1,mol in (enumerate(tqdm(mols))):
        index = 0
        res_temp = []
        while(len(res_temp)<20 and index <100):
            random_smiles = MolToSmiles(mol, doRandom=True) # e.g 'C(C(=O)O)N' 
            res_temp.append(random_smiles)
            res_temp = list(set(res_temp))
            index+=1
        res += res_temp
        res_labels += [labels[index1]]*len(res_temp)
        len_list.append(len(res_temp))
        
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        tmp = MolFromSmiles(smiles)
        for atom in tmp.GetAtoms():
            atom.SetAtomMapNum(atom.GetIdx())
        smiles_1 = MolToSmiles(tmp) # 'C([C:1](=[O:2])[OH:3])[NH2:4]' 
        ais_tokens_1 = atomInSmiles.encode(smiles_1, with_atomMap=True).split(' ')
        res_train.append(ais_tokens_1)

    return res_train,res_labels,len_list

In [8]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [9]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [36]:
len(encode(tox_train[:5],tox_label[:5])[0])

100%|██████████| 100/100 [00:00<00:00, 458.74it/s]


100

In [16]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [38]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open('./AIS_Tox_data/'+tox,'wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!
100%|██████████| 268795/268795 [06:41<00:00, 669.20it/s]
Found local copy...
Loading...
Done!
100%|██████████| 141234/141234 [01:48<00:00, 1297.08it/s]
Found local copy...
Loading...
Done!
[13:46:48] WARNING: not removing hydrogen atom without neighbors
[13:46:48] WARNING: not removing hydrogen atom without neighbors
 86%|████████▋ | 11293/13092 [00:14<00:01, 1198.55it/s][13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:

In [27]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label))
        with open('./AIS_Tox_data/'+tox+'_'+label,'wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
  4%|▍         | 257369/6137266 [05:12<4:55:06, 332.08it/s]